In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import os
from torch.utils.data import DataLoader, TensorDataset

In [2]:
work_dir = os.getcwd()#.replace("GitHub/RocketPill/ipynb-notebooks", "Data/")
# work_dir = os.getcwd().replace("GitHub/ipynb-notebooks", "Data/")
work_dir

'/Users/rckyi/Documents/GitHub/RocketPill/ipynb-notebooks'

In [3]:
data_dir = os.getcwd().replace("GitHub/RocketPill/ipynb-notebooks", "Data/")
data_dir

'/Users/rckyi/Documents/Data/'

In [4]:
thrust_curves_data_path = data_dir + "thrust_curves_normalized.csv"
thrust_curves_data_path

'/Users/rckyi/Documents/Data/thrust_curves_normalized.csv'

In [5]:
models_path = os.getcwd().replace("GitHub/RocketPill/ipynb-notebooks", "Deep_Learning_Models/") 
models_path

'/Users/rckyi/Documents/Deep_Learning_Models/'

In [6]:
def fetch_impulse_classes():
   
    # URL of the Wikipedia page
    url = "https://en.wikipedia.org/wiki/Model_rocket_motor_classification"
    
    # Send a request to the webpage
    response = requests.get(url)
    # print(f'response json {response.json()}')
    soup = BeautifulSoup(response.text, 'html.parser')
    # print(f'soup {soup}')
    
    # Find all tables
    tables = soup.find_all('table', {'class': 'wikitable'})
    
    # Look for the table that contains the header "Total impulse (N·s)"
    target_table = None
    for table in tables:
        # print(f'table text {table.}')
        if 'impulse' in table.text:
            print('found')
            target_table = table
            break
    # print(target_table)
    # Parse the table into a DataFrame
    df = pd.read_html(str(tables))[0]
    
    # Display the resulting DataFrame
    # print(df['Class  (Base 26)'])
    ls = df['Class  (Base 26)'].tolist()
    
    return [x for x in ls if len(x) < 10]

In [7]:
# 1. ThrustCurve.org API access
BASE_URL = "https://www.thrustcurve.org/api/v1"

def search_motors(impulse_class="F"):
    response = requests.get(f"{BASE_URL}/search.json", params={"impulseClass": impulse_class})
    # response = requests.get(f"{BASE_URL}/search.json", params={})
    response.raise_for_status()
    data = response.json()
    print(f'motor results {data.get("results", [])[0]}')
    return data.get("results", [])  # not "motors"

def download_thrust_curve(motor_id):
    response = requests.get(f"{BASE_URL}/download.json", params={"motorId": motor_id, "data": "samples"})
    response.raise_for_status()
    data = response.json()
    print(f'thrust data {data}')
    # print(f'thrust data {data["results"][0]}')
    if "samples" in str(data["results"]):
        print(f'samples found')
        return data["results"][0]["samples"]
    return []

In [8]:
def normalize_curve(curve):
    # print(f'curve time: {[x.get("time") for x in curve]}')
    if not curve:
        print(f'No curve')
        return None
    curve = np.array(curve)
    time = np.asarray([x.get("time") for x in curve]) # curve[:, 0]
    thrust = np.asarray([x.get("thrust") for x in curve]) # curve[:, 1]
    print(f'numpy time {time}')
    print(f'numpy thrust {thrust}')
    time = (time - time.min()) / (time.max() - time.min())
    thrust = thrust / thrust.max()
    return pd.DataFrame({"time": time, "thrust": thrust})

In [9]:
def collect_data(impulse_class, max_motors):
    motors = search_motors(impulse_class)
    print(f'num classes {len(classes)}')
    dataset = []
    for motor in tqdm(motors[:max_motors]):
        # print(f'motor {motor}')
        motor_id = motor["motorId"]
        motor_name = motor.get("designation", "unknown")
        samples = download_thrust_curve(motor_id)
        print(f'motor_id and motor_name {motor_id} {motor_name}')
        norm_df = normalize_curve(samples)
        # print(f'norm df {norm_df}')
        if norm_df is not None:
            norm_df["motorId"] = motor_id       
            norm_df["motor_name"] = motor_name
            norm_df["impulse_class"]=impulse_class
            dataset.append(norm_df)
            # print(f'norm_df {norm_df["motor_name"]}')
    final_df = pd.concat(dataset, ignore_index=True)
    print(f'final_df: {final_df.columns.tolist()}')
    # print(f'final_df rows: {final_df.shape[0]}')
    thrust_curves_data_path = data_dir + impulse_class +'_' + "thrust_curves_normalized.csv"
    final_df.to_csv(thrust_curves_data_path, index=False)
    print("Saved dataset to thrust_curves_normalized.csv")

In [10]:
if __name__ == "__main__":
    classes = fetch_impulse_classes()
    print(f'classes: {classes}')
    for c in classes[0:6]:
        print(f'IMPULSE CLASS {c} ===============================================')
        try:
            collect_data(c, max_motors=20)
        except:
            print(f'Data not found for {c} impulse class')
            pass
        
    # collect_data("F", max_motors=30)

classes: ['Micro', '1/4A', '1/2A', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ']
IMPULSE CLASS Micro ===============================================
Data not found for Micro impulse class
IMPULSE CLASS 1/4A ===============================================
Data not found for 1/4A impulse class
IMPULSE CLASS 1/2A ===============================================
Data not found for 1/2A impulse class
IMPULSE CLASS A ===============================================
motor results {'motorId': '5f4294d20002310000000002', 'manufacturer': 'Quest Aerospace', 'manufacturerAbbrev': 'Quest', 'designation': 'MICRO_MAXX', 'commonName': '1/8A0.3', 'impulseClass': 'A', 'diameter': 6, 'length': 26, 'type': 'SU', 'certOrg': 'National Association of Rocketry', 'avgThrustN': 0.163, 'maxThrustN': 1.898, 'totImpulseNs': 0.135, 'burnTimeS': 0.827, 'dataFiles': 2, 'infoUrl

  6%|██████                                                                                                 | 1/17 [00:00<00:03,  4.90it/s]

thrust data {'results': [{'motorId': '5f4294d20002310000000002', 'simfileId': '5f4294d20002e90000000820', 'format': 'RASP', 'source': 'cert', 'samples': [{'time': 0.016, 'thrust': 0.0530363}, {'time': 0.02, 'thrust': 0.0985533}, {'time': 0.024, 'thrust': 0.176654}, {'time': 0.03, 'thrust': 0.415869}, {'time': 0.032, 'thrust': 0.434918}, {'time': 0.038, 'thrust': 0.657991}, {'time': 0.052, 'thrust': 1.247306}, {'time': 0.058, 'thrust': 1.453536}, {'time': 0.064, 'thrust': 1.700872}, {'time': 0.066, 'thrust': 1.755612}, {'time': 0.07, 'thrust': 1.951716}, {'time': 0.074, 'thrust': 2.06541}, {'time': 0.076, 'thrust': 2.09889}, {'time': 0.078, 'thrust': 2.09889}, {'time': 0.08, 'thrust': 2.02811}, {'time': 0.082, 'thrust': 1.911814}, {'time': 0.084, 'thrust': 1.619262}, {'time': 0.086, 'thrust': 1.151861}, {'time': 0.088, 'thrust': 0.728272}, {'time': 0.09, 'thrust': 0.433614}, {'time': 0.092, 'thrust': 0.292953}, {'time': 0.098, 'thrust': 0.217459}, {'time': 0.154, 'thrust': 0.224377}, {'

 12%|████████████                                                                                           | 2/17 [00:00<00:03,  4.86it/s]

thrust data {'results': [{'motorId': '5f4294d20002310000000001', 'simfileId': '5f4294d20002e90000000821', 'format': 'RASP', 'source': 'cert', 'samples': [{'time': 0.026, 'thrust': 0.0394607}, {'time': 0.036, 'thrust': 0.1214556}, {'time': 0.04, 'thrust': 0.206028}, {'time': 0.05, 'thrust': 0.444676}, {'time': 0.056, 'thrust': 0.680251}, {'time': 0.06, 'thrust': 0.899565}, {'time': 0.066, 'thrust': 1.358816}, {'time': 0.068, 'thrust': 1.475611}, {'time': 0.07, 'thrust': 1.56653}, {'time': 0.074, 'thrust': 1.624927}, {'time': 0.088, 'thrust': 1.190166}, {'time': 0.112, 'thrust': 0.830955}, {'time': 0.126, 'thrust': 0.439917}, {'time': 0.134, 'thrust': 0.319751}, {'time': 0.144, 'thrust': 0.248959}, {'time': 0.174, 'thrust': 0.1656754}, {'time': 0.206, 'thrust': 0.1918503}, {'time': 0.258, 'thrust': 0.1262147}, {'time': 0.342, 'thrust': 0.1601231}, {'time': 0.366, 'thrust': 0.1386081}, {'time': 0.384, 'thrust': 0.1794569}, {'time': 0.494, 'thrust': 0.218422}, {'time': 0.546, 'thrust': 0.1

...

thrust data {'results': [{'motorId': '5f7d2f71606b5d00041d9aba', 'simfileId': '5f923ddc1bca580004171646', 'format': 'RockSim', 'source': 'user', 'license': 'PD', 'samples': [{'time': 0, 'thrust': 0}, {'time': 0.013, 'thrust': 0.043}, {'time': 0.022, 'thrust': 0.937}, {'time': 0.03, 'thrust': 2.428}, {'time': 0.039, 'thrust': 6.218}, {'time': 0.044, 'thrust': 12.436}, {'time': 0.052, 'thrust': 20.06}, {'time': 0.059, 'thrust': 21.721}, {'time': 0.071, 'thrust': 22.019}, {'time': 0.08, 'thrust': 21.848}, {'time': 0.088, 'thrust': 22.189}, {'time': 0.122, 'thrust': 21.38}, {'time': 0.132, 'thrust': 21.422}, {'time': 0.143, 'thrust': 21.082}, {'time': 0.152, 'thrust': 21.55}, {'time': 0.167, 'thrust': 21.678}, {'time': 0.177, 'thrust': 21.721}, {'time': 0.187, 'thrust': 21.934}, {'time': 0.197, 'thrust': 21.21}, {'time': 0.218, 'thrust': 21.252}, {'time': 0.225, 'thrust': 21.976}, {'time': 0.232, 'thrust': 22.061}, {'time': 0.241, 'thrust': 21.082}, {'time': 0.247, 'thrust': 20.954}, {'tim

 92%|██████████████████████████████████████████████████████████████████████████████████████████████▏       | 12/13 [00:02<00:00,  4.81it/s]

thrust data {'results': [{'motorId': '5f4294d20002310000000018', 'simfileId': '5f4294d20002e90000000351', 'format': 'RockSim', 'source': 'user', 'samples': [{'time': 0, 'thrust': 0}, {'time': 0.008, 'thrust': 13.958}, {'time': 0.016, 'thrust': 21.1}, {'time': 0.022, 'thrust': 15.511}, {'time': 0.03, 'thrust': 12.831}, {'time': 0.052, 'thrust': 14.8}, {'time': 0.081, 'thrust': 15.927}, {'time': 0.092, 'thrust': 14.658}, {'time': 0.114, 'thrust': 16.069}, {'time': 0.125, 'thrust': 14.658}, {'time': 0.136, 'thrust': 15.369}, {'time': 0.168, 'thrust': 14.8}, {'time': 0.214, 'thrust': 13.816}, {'time': 0.225, 'thrust': 12.973}, {'time': 0.247, 'thrust': 13.958}, {'time': 0.252, 'thrust': 12.831}, {'time': 0.285, 'thrust': 12.547}, {'time': 0.307, 'thrust': 12.405}, {'time': 0.317, 'thrust': 12.831}, {'time': 0.328, 'thrust': 11.562}, {'time': 0.347, 'thrust': 11.988}, {'time': 0.393, 'thrust': 11.42}, {'time': 0.442, 'thrust': 10.719}, {'time': 0.464, 'thrust': 11.136}, {'time': 0.488, 'thr

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:02<00:00,  4.80it/s]

thrust data {'results': [{'motorId': '5f76974c9503a10004898858', 'simfileId': '5f8b0656d5fa3b000447e82f', 'format': 'RockSim', 'source': 'mfr', 'license': 'PD', 'samples': [{'time': 0, 'thrust': 0}, {'time': 0.046, 'thrust': 0.953}, {'time': 0.168, 'thrust': 5.259}, {'time': 0.235, 'thrust': 10.023}, {'time': 0.291, 'thrust': 15}, {'time': 0.418, 'thrust': 9.87}, {'time': 0.505, 'thrust': 7.546}, {'time': 0.582, 'thrust': 6.631}, {'time': 0.679, 'thrust': 6.136}, {'time': 0.786, 'thrust': 5.716}, {'time': 1.26, 'thrust': 5.678}, {'time': 1.357, 'thrust': 5.488}, {'time': 1.423, 'thrust': 4.992}, {'time': 1.469, 'thrust': 4.116}, {'time': 1.618, 'thrust': 1.22}, {'time': 1.701, 'thrust': 0}], 'infoUrl': '/simfiles/5f8b0656d5fa3b000447e82f/', 'dataUrl': '/simfiles/5f8b0656d5fa3b000447e82f/download/data.rse'}, {'motorId': '5f76974c9503a10004898858', 'simfileId': '5f8b0644d5fa3b000447e82e', 'format': 'RASP', 'source': 'mfr', 'license': 'PD', 'samples': [{'time': 0.046, 'thrust': 0.953}, {'